In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import os
import numpy as np
import pandas as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = "data"

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,              
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 396 images belonging to 2 classes.


роздіємо на тренувальні дані і дані для тестування

In [ ]:
import os
import shutil
import random

data_dir = "data" 
healthy_dir = os.path.join(data_dir, "DENTAL_HEALTHY_TEETH")  # папка зі здоровими зубами
diseased_dir = os.path.join(data_dir, "DENTAL_NOT_HEALTHY_TEETH")  # папка з хворими зубами

train_dir = "train"
test_dir = "test"

for folder in [train_dir, test_dir]:
    os.makedirs(os.path.join(folder, "healthy"), exist_ok=True)
    os.makedirs(os.path.join(folder, "diseased"), exist_ok=True)

def split_data(source_dir, train_dest, test_dest, train_ratio=0.75):
    files = os.listdir(source_dir)
    random.shuffle(files)
    n_train = int(len(files) * train_ratio)
    train_files = files[:n_train]
    test_files = files[n_train:]

    for f in train_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(train_dest, f))
    for f in test_files:
        shutil.copy(os.path.join(source_dir, f), os.path.join(test_dest, f))

split_data(
    healthy_dir,
    os.path.join(train_dir, "healthy"),
    os.path.join(test_dir, "healthy")
)

split_data(
    diseased_dir,
    os.path.join(train_dir, "diseased"),
    os.path.join(test_dir, "diseased")
)

print("Розподіл завершено!")


Розподіл завершено!


In [ ]:
from tensorflow.keras import backend as K

img_width, img_height = 150, 150

train_data_dir = 'train'  # папка з train/healthy і train/diseased
test_data_dir = 'test'    # папка з test/healthy і test/diseased

epochs = 20
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


Створення моделі

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('sigmoid'))


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Компіляція

In [6]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


генератори даних

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 296 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


Навчання моделі

In [ ]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = max(1, test_generator.samples // batch_size)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 170ms/step - accuracy: 0.5143 - loss: 0.8540 - val_accuracy: 0.5521 - val_loss: 0.6889
Epoch 2/20
 1/18 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.5000 - loss: 0.7336

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.5000 - loss: 0.7336 - val_accuracy: 0.5625 - val_loss: 0.6870
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - accuracy: 0.5357 - loss: 0.6930 - val_accuracy: 0.5521 - val_loss: 0.6889
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6250 - loss: 0.6907 - val_accuracy: 0.5521 - val_loss: 0.6886
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.5643 - loss: 0.6884 - val_accuracy: 0.5521 - val_loss: 0.7706
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.5000 - loss: 0.8971 - val_accuracy: 0.5729 - val_loss: 0.6854
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 144ms/step - accuracy: 0.5714 - loss: 0.6908 - val_accuracy: 0.5625 - val_loss: 0.6871
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.4375 - loss: 0.6976 - val_accuracy: 0.5625 - val_loss: 0.6874
Epoch 9/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 145ms/step - accuracy: 0.5694 - loss: 0.6895 - val_accuracy: 0.5625 - val_loss

збереження і оцінка

In [ ]:
model.save_weights('teeth_model.weights.h5')


scores = model.evaluate(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - accuracy: 0.5600 - loss: 0.6868

compile_metrics: 56.00%


Дає точість +- 50/50, що не дуже добре, тому візьмеммо більше поколінь


In [ ]:
from tensorflow.keras import backend as K

img_width, img_height = 150, 150

train_data_dir = 'train'  
test_data_dir = 'test'   

epochs = 5
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


In [17]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 296 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [ ]:
steps_per_epoch = train_generator.samples // batch_size
validation_steps = max(1, test_generator.samples // batch_size)  

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=validation_steps)


Epoch 1/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - accuracy: 0.6214 - loss: 0.6220 - val_accuracy: 0.2604 - val_loss: 1.8719
Epoch 2/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6250 - loss: 0.5329 - val_accuracy: 0.2604 - val_loss: 2.0486
Epoch 3/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 141ms/step - accuracy: 0.6536 - loss: 0.5686 - val_accuracy: 0.2917 - val_loss: 3.2375
Epoch 4/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.7500 - loss: 0.5110 - val_accuracy: 0.2812 - val_loss: 3.2722
Epoch 5/5
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 143ms/step - accuracy: 0.6500 - loss: 0.5418 - val_accuracy: 0.2812 - val_loss: 2.2320


In [ ]:
model.save_weights('teeth_model_2.weights.h5')


scores = model.evaluate(test_generator)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.2800 - loss: 2.8376

compile_metrics: 28.00%


бачимо що модель все одно дуже погано працює
Одна з причин це те що у нас пащі з зубами які у різному стані. Тому варто розбивати на категорії і ідентифікувати хвороби

In [ ]:
import os
import shutil
import random

base_dir = 'data_clas'
train_dir = 'train_clas'
test_dir = 'test_clas'
test_ratio = 0.75

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)

    if not os.path.isdir(class_path):
        continue

    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    random.shuffle(files)

    test_size = int(len(files) * test_ratio)
    test_files = files[:test_size]
    train_files = files[test_size:]

    for f in train_files:
        src = os.path.join(class_path, f)
        dst = os.path.join(train_dir, class_name, f)
        shutil.move(src, dst)

    for f in test_files:
        src = os.path.join(class_path, f)
        dst = os.path.join(test_dir, class_name, f)
        shutil.move(src, dst)


✅ Дані успішно розділено без повторів!
Тренувальні дані збережено у 'train_clas', тестові у 'test_clas'


In [ ]:
train_dir = 'train_clas'
test_dir = 'test_clas'

img_height, img_width = 150, 150
batch_size = 32


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 106 images belonging to 6 classes.
Found 24 images belonging to 6 classes.
Found 387 images belonging to 6 classes.


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.4),
    Dense(train_generator.num_classes, activation='softmax')
])


In [26]:
from tensorflow.keras.optimizers import Adam

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 74, 74, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 36, 36, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 17, 17, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     9,470,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,565,894 (36.49 MB)

 Trainable params: 9,565,446 (36.49 MB)

 Non-trainable params: 448 (1.75 KB)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"\nТочність на тестових даних: {test_acc:.4f}")
model.save('teeth_classifier.keras')


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 0.1705 - loss: 1.7812

🎯 Точність на тестових даних: 0.1705


In [ ]:
import numpy as np

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

print("Класи:", test_generator.class_indices)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 156ms/step
Класи: {'BDC-BDR': 0, 'Caries': 1, 'Fractured Teeth': 2, 'Healthy Teeth': 3, 'Impacted teeth': 4, 'Infection': 5}
